In [ ]:
from flask_wtf import Form
from wtforms import TextField

In [98]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
res = es.search(index='artist', body={'query': {'match_all':{}}})
print "Got " + str(res['hits']['total']) + " hits"
print res['hits']['hits']

Got 2 hits
[{u'_score': 1.0, u'_type': u'bieber', u'_id': u'AVRFA3AU3VP-SR7EVm0P', u'_source': {u'best_song.title': u'Eenie Meenie', u'best_song.release_year': 2010, u'talent': 5}, u'_index': u'artist'}, {u'_score': 1.0, u'_type': u'plant', u'_id': u'AVRFA5wn3VP-SR7EVm0Q', u'_source': {u'best_song.title': u'Bron-Y-Aur Stomp', u'best_song.release_year': 1970, u'talent': 90}, u'_index': u'artist'}]


In [90]:
%%writefile form.py
from flask_wtf import Form
from wtforms import TextField, TextAreaField, SubmitField

class ContactForm(Form):
  name = TextField("Name")
  salary = TextField("Salary")
  submit = SubmitField("Form")

Overwriting form.py


In [123]:
%%writefile routes.py
from flask import Flask, render_template, request
from form import ContactForm
from elasticsearch import Elasticsearch

app = Flask(__name__)

app.secret_key = "nihaoma"

@app.route('/')
@app.route('/form', methods = ["GET", "POST"])
def form():
    form = ContactForm()
    
    if request.method == "POST":
        print "POST used."
        es = Elasticsearch()
        
        res = es.search(index='va', 
                        doc_type="plan",
                        body = {
                                    'query': {
                                        'match_all': {}
                                }
                }
                       )
        print "Got " + str(res['hits']['total']) + " hits"
        return render_template('resultsrefined.html', res=res['hits']['hits'])  
    elif request.method == "GET":
#         return render_template("basicform.html")
#         return "Hello world."
        print "Calling render_template"
        return render_template('form.html', form=form)
    
if __name__=="__main__":
    app.run(debug=True)

Overwriting routes.py


In [145]:
%%writefile templates/layout.html
<!DOCTYPE html>
<html>
  <head>
    <title>Innocent Bayestanders</title>    
  </head>
  <body>
   
    <header>
      <div class="container">
        <h1 class="logo">Innocent Bayestanders App</h1>
      </div>
    </header> 
     
    <div class="container">
      {% block content %}
      {% endblock %}
    </div>
     
  </body>
</html>

Overwriting templates/layout.html


In [89]:
%%writefile templates/form.html
{% extends "layout.html" %}
 
{% block content %}
  <h2>Contact</h2>
  <form action="{{ url_for('form') }}" method=POST> 
    {{ form.name.label }}
    {{ form.name }}
     <br>
    {{ form.salary.label }}
    {{ form.salary }}
      <br>
    {{ form.submit }}
  </form>
{% endblock %}

Overwriting templates/form.html


In [ ]:
%%writefile templates/results.html
{% extends "layout.html" %}
 
{% block content %}
  <h2>Results</h2>
    
    {% for result in res %}
    <p>{{ result }}</p>
    {% endfor %}
{% endblock %}

In [143]:
%%writefile templates/resultsrefined.html
<html>
{% raw %}
	<head>
	 	<title>Response Page</title>
	 	<meta charset="utf-8" />
	 	<!-- Snowplow starts plowing -->
		<script type="text/javascript">
		;(function(p,l,o,w,i,n,g){if(!p[i]){p.GlobalSnowplowNamespace=p.GlobalSnowplowNamespace||[];
		p.GlobalSnowplowNamespace.push(i);p[i]=function(){(p[i].q=p[i].q||[]).push(arguments)
		};p[i].q=p[i].q||[];n=l.createElement(o);g=l.getElementsByTagName(o)[0];n.async=1;
		n.src=w;g.parentNode.insertBefore(n,g)}}(window,document,"script","//d1fc8wv8zag5ca.cloudfront.net/2.6.1/sp.js","snowplow"));
​
		window.snowplow('newTracker', 'cf', 'dfj7gak3gw4nh.cloudfront.net', { // Initialise a tracker
		  appId: '{{MY-SITE-ID}}',
		  cookieDomain: '{{MY-COOKIE-DOMAIN}}'
		});
​
		window.snowplow('trackPageView');
		window.snowplow('enableLinkClickTracking');
		</script>
		<!-- Snowplow stops plowing -->
		
		<link rel="stylesheet" href="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css">
		<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js"></script>
		<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
		<style>
			/* Icon when the collapsible content is shown */
			.btn:after {
			font-family: "Glyphicons Halflings";
			content: "\e114";
			float: right;
			margin-left: 15px;
			}
			/* Icon when the collapsible content is hidden */
			.btn.collapsed:after {
			content: "\e080";
			}
		</style>
​
	</head>
{% endraw %}
​
	<body>
    {% set i = 0 %}
    {% for result in res %}
        {% set i = i + 1 %}
		<div class="container">
			<h3>Plan {{i}}</h3>
	  		<button type="button" class="btn btn-sm btn-info collapsed" data-toggle="collapse" data-target={{'#demo' ~ i}}>More</button>
	  		<div id={{'demo' ~ i}} class="collapse">
	  			<br>
                <p>Plan ID: {{ result['_source']['plan_id'] }}</p>
			    <p>Contact: {{ result['_source']['plan_contact'] }}</p>
				<p>(Show similar plans to this one)</p>
		 	</div>
		 	<br>
		 	<a id={{'plan' ~ i}} href="http://google.com">SIGN UP</a>
		</div>
    {% endfor %}
​
	</body>
​
</html>

Overwriting templates/resultsrefined.html
